In [1]:
import os.path, pickle, math
import numpy as np
import scipy as sp
import pandas as pd
from constants import PROCESSED_PATH, RAW_PATH
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from sklearn.utils import shuffle, resample
from sklearn.preprocessing import normalize, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
import sklearn.metrics as skm

In [3]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [4]:
datafn = 'HOUR_00003.csv'

In [5]:
df = pd.read_csv(os.path.join(PROCESSED_PATH, datafn), na_values=['?', '!'])
df.replace('!.+', np.nan, regex=True, inplace=True)

In [6]:
df.head()

,SUBJECT_ID,HADM_ID,AGE,GENDER,ETHNICITY,P WEIGHT,P HEIGHT,P SYSTOLIC BP,P DIASTOLIC BP,P TEMPERATURE,...,MACROCYTES,PEEP,ATYPICAL LYMPHOCYTES,METAMYELOCYTES,MYELOCYTES,ANISOCYTOSIS,MICROCYTES,SODIUM.2,TSTAGE,STAGE
0,3,145834,76,1,WHITE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,2
1,4,185777,47,0,WHITE,53.6,NaN,116.0,63.00,37.444422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,9,150750,41,1,UNKNOWN/NOT SPECIFIED,104.0,182.88,168.0,88.75,35.277789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,1
3,11,194540,50,0,WHITE,NaN,NaN,110.5,52.00,37.055553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,13,143045,39,0,WHITE,73.5,144.78,149.0,72.50,37.277789,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [7]:
check_for_nan_columns = set(df.columns) - {'SUBJECT_ID', 'HADM_ID', 'AGE', 'GENDER', 'ETHNICITY','P TSTAGE','P STAGE','TSTAGE','STAGE'}
df = df.astype({k: np.float64 for k in check_for_nan_columns}, inplace=True)

In [8]:
# drop rows where all features=nan
row_nan_bool = np.logical_not(np.all(np.isnan(df.iloc[:,5:-1]), axis=1))
df = df[row_nan_bool]

In [9]:
df.sort_values(['SUBJECT_ID', 'HADM_ID'], inplace=True)

In [10]:
ids_fn = os.path.join(RAW_PATH, 'd_ids_split.pickle')

In [11]:
split_ids = pickle.load(open(ids_fn, 'rb'))

In [12]:
split_df = {}
for dataset in split_ids:
    split_df[dataset] = df[(df['SUBJECT_ID'].isin(split_ids[dataset][:,0])) & (df['HADM_ID'].isin(split_ids[dataset][:,1]))]
devel = split_df['devel']

In [13]:
for k in split_df:
    print(k, split_df[k].shape)
print('total', df.shape)

test (5579, 205)
devel (16609, 205)
valid (5492, 205)
total (27680, 205)


In [14]:
#drop columns where all rows=nan
check_nan = devel.isna().sum()
devel.drop(labels=check_nan[(check_nan == devel.shape[0])].keys(), axis=1, inplace=True)

/home/ram/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [15]:
devel.head()

,SUBJECT_ID,HADM_ID,AGE,GENDER,ETHNICITY,P WEIGHT,P HEIGHT,P SYSTOLIC BP,P DIASTOLIC BP,P TEMPERATURE,...,WEIGHT,HEIGHT,SYSTOLIC BP,DIASTOLIC BP,TEMPERATURE,RESPIRATORY RATE,HEART RATE,SPO2,TSTAGE,STAGE
1,4,185777,47,0,WHITE,53.600000,NaN,116.000000,63.00000,37.444422,...,53.6,NaN,116.00,63.0,37.444422,NaN,105.5,98.000000,0,0
2,9,150750,41,1,UNKNOWN/NOT SPECIFIED,104.000000,182.88,168.000000,88.75000,35.277789,...,104.0,182.88,160.75,82.5,36.055578,10.625,87.0,98.800000,37,1
4,13,143045,39,0,WHITE,73.500000,144.78,149.000000,72.50000,37.277789,...,73.5,144.78,147.00,72.0,37.277789,15.000,77.0,98.000000,0,0
6,18,188822,50,1,WHITE,NaN,NaN,151.400000,82.20000,37.666683,...,NaN,NaN,152.00,75.0,37.666683,21.000,116.0,98.666667,0,0
7,21,111970,87,1,WHITE,64.942857,175.26,130.144828,51.62069,36.383338,...,64.0,NaN,115.00,32.0,37.611106,14.000,68.0,100.000000,8,3


In [16]:
devel = devel[devel.columns[[0,1,4,2,3] + list(range(5,len(devel.columns)))]]

In [17]:
devel.head()

,SUBJECT_ID,HADM_ID,ETHNICITY,AGE,GENDER,P WEIGHT,P HEIGHT,P SYSTOLIC BP,P DIASTOLIC BP,P TEMPERATURE,...,WEIGHT,HEIGHT,SYSTOLIC BP,DIASTOLIC BP,TEMPERATURE,RESPIRATORY RATE,HEART RATE,SPO2,TSTAGE,STAGE
1,4,185777,WHITE,47,0,53.600000,NaN,116.000000,63.00000,37.444422,...,53.6,NaN,116.00,63.0,37.444422,NaN,105.5,98.000000,0,0
2,9,150750,UNKNOWN/NOT SPECIFIED,41,1,104.000000,182.88,168.000000,88.75000,35.277789,...,104.0,182.88,160.75,82.5,36.055578,10.625,87.0,98.800000,37,1
4,13,143045,WHITE,39,0,73.500000,144.78,149.000000,72.50000,37.277789,...,73.5,144.78,147.00,72.0,37.277789,15.000,77.0,98.000000,0,0
6,18,188822,WHITE,50,1,NaN,NaN,151.400000,82.20000,37.666683,...,NaN,NaN,152.00,75.0,37.666683,21.000,116.0,98.666667,0,0
7,21,111970,WHITE,87,1,64.942857,175.26,130.144828,51.62069,36.383338,...,64.0,NaN,115.00,32.0,37.611106,14.000,68.0,100.000000,8,3


In [18]:
data3 = devel.iloc[:,3:-2]
data3.head()

,AGE,GENDER,P WEIGHT,P HEIGHT,P SYSTOLIC BP,P DIASTOLIC BP,P TEMPERATURE,P RESPIRATORY RATE,P HEART RATE,P SPO2,...,P TSTAGE,P STAGE,WEIGHT,HEIGHT,SYSTOLIC BP,DIASTOLIC BP,TEMPERATURE,RESPIRATORY RATE,HEART RATE,SPO2
1,47,0,53.600000,NaN,116.000000,63.00000,37.444422,NaN,105.500000,98.000000,...,0,0,53.6,NaN,116.00,63.0,37.444422,NaN,105.5,98.000000
2,41,1,104.000000,182.88,168.000000,88.75000,35.277789,12.800000,84.000000,98.625000,...,37,1,104.0,182.88,160.75,82.5,36.055578,10.625,87.0,98.800000
4,39,0,73.500000,144.78,149.000000,72.50000,37.277789,17.000000,78.500000,98.000000,...,0,0,73.5,144.78,147.00,72.0,37.277789,15.000,77.0,98.000000
6,50,1,NaN,NaN,151.400000,82.20000,37.666683,22.000000,104.000000,99.200000,...,0,0,NaN,NaN,152.00,75.0,37.666683,21.000,116.0,98.666667
7,87,1,64.942857,175.26,130.144828,51.62069,36.383338,19.422535,76.583333,98.887324,...,8,3,64.0,NaN,115.00,32.0,37.611106,14.000,68.0,100.000000


In [19]:
# calculate Kruskal-Wallis H-test for each feature
dfs_by_class = [data3.loc[devel['STAGE'] == c] for c in [0,1,2,3]]
kruskals = {}
for col in data3.columns:
    col_in_classes = [np.asarray(c[col].dropna()) for c in dfs_by_class]
    try:
        kruskals[col] = sp.stats.kruskal(*col_in_classes)[1]
    except ValueError:
        kruskals[col] = 0
           
devel_kruskal = devel[list(devel.columns[:3])+[k for k, v in kruskals.items() if v > 0.05]+list(devel.columns[-2:])]

In [20]:
devel_kruskal.shape

(16609, 11)

In [21]:
means = devel_kruskal.mean()
devel_kruskal.fillna(means, inplace=True)
print(devel_kruskal.head())

/home/ram/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


   SUBJECT_ID  HADM_ID              ETHNICITY       P PO2  P FREE CALCIUM  \
1           4   185777                  WHITE   99.508842        1.119181   
2           9   150750  UNKNOWN/NOT SPECIFIED   99.508842        1.119181   
4          13   143045                  WHITE   99.508842        1.119181   
6          18   188822                  WHITE   99.508842        1.160000   
7          21   111970                  WHITE  106.090909        1.231667   

   P CREATINE KINASE (CK)  P LIPASE  P LENGTH OF URINE COLLECTION  HEART RATE  \
1               98.652439  268.2714                     23.558282       105.5   
2              112.000000  268.2714                     23.558282        87.0   
4               74.000000  268.2714                     23.558282        77.0   
6              177.000000  268.2714                     23.558282       116.0   
7               98.652439  268.2714                     23.558282        68.0   

   TSTAGE  STAGE  
1       0      0  
2      37   

In [22]:
# calculate VIFs
features = devel_kruskal[devel_kruskal.columns[3:-2]]
print(features.columns)
done = -1
while done != 0:
    vifs = {}
    for i, n in enumerate(features):
        if i in range(3,features.shape[1]):
            vifs[n] = variance_inflation_factor(np.asarray(features), i)

    drop_vifs = [k for k,v in vifs.items() if v >= 5 or np.isnan(v)]
    print(drop_vifs)
    features.drop(labels=drop_vifs, axis=1, inplace=True)
    done = len(drop_vifs)

Index(['P PO2', 'P FREE CALCIUM', 'P CREATINE KINASE (CK)', 'P LIPASE',
       'P LENGTH OF URINE COLLECTION', 'HEART RATE'],
      dtype='object')
['P LENGTH OF URINE COLLECTION', 'HEART RATE']
[]


In [23]:
devel_vif = devel_kruskal[list(devel_kruskal.columns[:3]) + list(features.columns) + list(devel_kruskal.columns[-2:])]
devel_vif.head()

,SUBJECT_ID,HADM_ID,ETHNICITY,P PO2,P FREE CALCIUM,P CREATINE KINASE (CK),P LIPASE,TSTAGE,STAGE
1,4,185777,WHITE,99.508842,1.119181,98.652439,268.2714,0,0
2,9,150750,UNKNOWN/NOT SPECIFIED,99.508842,1.119181,112.000000,268.2714,37,1
4,13,143045,WHITE,99.508842,1.119181,74.000000,268.2714,0,0
6,18,188822,WHITE,99.508842,1.160000,177.000000,268.2714,0,0
7,21,111970,WHITE,106.090909,1.231667,98.652439,268.2714,8,3


In [24]:
devel_dist = {}
for s in [0,1,2,3]:
    devel_dist[s] = devel_vif[(devel_vif['STAGE'] == s)].shape[0] / devel_vif.shape[0]
devel_dist

{0: 0.7371906797519417,
 1: 0.10614726955265218,
 2: 0.06562706966102716,
 3: 0.09103498103437895}

In [25]:
counts = {subset: dict() for subset in ['test', 'valid']}
for subset in counts:
    for s in [0,1,2,3]:
        counts[subset][s] = split_df[subset][(split_df[subset]['STAGE'] == s)].shape[0]

dists = {subset: dict() for subset in ['test', 'valid']}
for subset in dists:
    for s in [0,1,2,3]:
        dists[subset][s] = counts[subset][s] / split_df[subset].shape[0]
ratios = {}
for subset in dists:
    print(subset, dists[subset])
    ratios[subset] = {c: dists[subset][c]/devel_dist[c] for c in devel_dist}
    print('ratio', ratios[subset])

test {0: 0.7397383043556193, 1: 0.10503674493636853, 2: 0.05968811614984764, 3: 0.09553683455816454}
ratio {0: 1.0034558556878863, 1: 0.9895378880590725, 2: 0.9095045148007517, 3: 1.0494519081855522}
valid {0: 0.7408958485069191, 1: 0.10342316096139839, 2: 0.06900946831755281, 3: 0.08667152221412965}
ratio {0: 1.0050260656526804, 1: 0.9743365175314044, 2: 1.0515396874185639, 3: 0.9520683283429097}


In [26]:
synthesize = {subset: dict() for subset in ['test', 'valid']}
for subset in ratios:
    for c in ratios[subset]:
        if ratios[subset][c] < 1:
            num_syn = math.ceil((1-ratios[subset][c])*counts[subset][c])
            print(subset, c, counts[subset][c], num_syn)
            synthesize[subset][c] = num_syn
synthesize

test 1 586 7
test 2 333 31
valid 1 568 15
valid 3 476 23


{'test': {1: 7, 2: 31}, 'valid': {1: 15, 3: 23}}

In [27]:
for subset in synthesize:
    for c in synthesize[subset]:
        split_df[subset] = split_df[subset].append(
            resample(
                split_df[subset][(split_df[subset]['STAGE'] == c)],
                n_samples=synthesize[subset][c],
            )
        )
        print(subset, split_df[subset].shape)


test (5586, 205)
test (5617, 205)
valid (5507, 205)
valid (5530, 205)


In [28]:
for subset in synthesize:
    for c in synthesize[subset]:
        print(subset, c, split_df[subset][(split_df[subset]['STAGE'] == c)].shape[0])
counts

test 1 593
test 2 364
valid 1 583
valid 3 499


{'test': {0: 4127, 1: 586, 2: 333, 3: 533},
 'valid': {0: 4069, 1: 568, 2: 379, 3: 476}}

In [29]:
test = split_df['test']
valid = split_df['valid']

test.fillna(means, inplace=True)
valid.fillna(means, inplace=True)

test = test[devel_vif.columns]
valid = valid[devel_vif.columns]

train = devel_vif.drop(['SUBJECT_ID','HADM_ID','ETHNICITY','TSTAGE'], axis=1)
testv = test.drop(['SUBJECT_ID','HADM_ID','ETHNICITY','TSTAGE'], axis=1)

x_train = train.values[:, :-2]
y_train = train.values[:, -1]
x_train = normalize(x_train, axis=0)
ohe = LabelBinarizer()
ohe.fit(y_train.reshape(-1, 1))

x_test = testv.values[:, :-2]
x_test = normalize(x_test, axis=0)
y_test = testv.values[:, -1]

ohe_y_train = ohe.transform(y_train.reshape(-1,1))
ohe_y_test = ohe.transform(y_test.reshape(-1,1))
print(ohe_y_train.shape)
print(ohe_y_test.shape)

(16609, 4)
(5617, 4)


In [30]:
def runModel(model, x, y, x_test, y_test):
    m = OneVsRestClassifier(model)
    m.fit(x,y)
    y_predict = m.predict(x_test)
    
    return m, y_predict

In [31]:
def runMetrics(y_test, y_predict):
    TP = {}
    FP = {}
    TN = {}
    FN = {}

    for i in range(y_predict.shape[1]): 
        TP[i] = np.sum(np.logical_and(y_predict[:,i]==1,y_test[:,i]==1))
        FP[i] = np.sum(np.logical_and(y_predict[:,i]==1,y_test[:,i]!=y_predict[:,i]))
        TN[i] = np.sum(np.logical_and(y_predict[:,i]==0,y_test[:,i]==0))
        FN[i] = np.sum(np.logical_and(y_predict[:,i]==0,y_test[:,i]!=y_predict[:,i]))
    
    out = {}
    for i in range(y_predict.shape[1]): 
        out[i] = [TP[i], FP[i], TN[i], FN[i]]
    return out

In [32]:
def multiclass_auc(y_test, y_score):
    n_classes = 4
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = skm.roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = skm.auc(fpr[i], tpr[i])
    roc_auc['avg'] = sum(roc_auc.values())/n_classes
    return roc_auc

In [33]:
models = {
    'SVC': SVC(),
    'SGDClassifier': SGDClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'GaussianNB': GaussianNB(),
    'KNeighborsClassifier': KNeighborsClassifier(),
#     'GaussianProcessClassifier': GaussianProcessClassifier(), # this one was giving me an out of memory error
    'MLPClassifier': MLPClassifier(),
}

In [34]:
rawScores = {}
accScores = {}
for n, m in models.items():
    print(n)
    m, y_predict = runModel(m, x_train, ohe_y_train, x_test, ohe_y_test)
    accScores[n] = skm.accuracy_score(ohe_y_test, y_predict)
    rawScores[n] = runMetrics(ohe_y_test, y_predict)
    if n in {'SVC', 'SGDClassifier', 'GradientBoostingClassifier'}:
        aucs = multiclass_auc(ohe_y_test, m.decision_function(x_test))
    else:
        aucs = multiclass_auc(ohe_y_test, m.predict_proba(x_test))
        
    for k in rawScores[n].keys():
        rawScores[n][k].append(aucs[k])
    

SVC


/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/svm/base

SGDClassifier
GradientBoostingClassifier


/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/ram/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they defau

DecisionTreeClassifier
GaussianNB
KNeighborsClassifier
MLPClassifier


In [35]:
print(rawScores)

{'SVC': {0: [4127, 1490, 0, 0, 0.48568178454863453], 1: [0, 0, 5024, 593, 0.5248810767338696], 2: [0, 0, 5253, 364, 0.5083908096472345], 3: [0, 0, 5084, 533, 0.48860033242649203]}, 'SGDClassifier': {0: [4127, 1490, 0, 0, 0.47441338183805126], 1: [0, 0, 5024, 593, 0.5092579228472305], 2: [0, 0, 5253, 364, 0.5381399012181423], 3: [0, 0, 5084, 533, 0.5179618432842321]}, 'GradientBoostingClassifier': {0: [548, 304, 1186, 3579, 0.4464472787649836], 1: [9, 36, 4988, 584, 0.508898266398857], 2: [273, 4264, 989, 91, 0.4731210632124396], 3: [42, 223, 4861, 491, 0.45182417561329885]}, 'DecisionTreeClassifier': {0: [258, 150, 1340, 3869, 0.4809220016164626], 1: [74, 542, 4482, 519, 0.5084535209074017], 2: [31, 318, 4935, 333, 0.5123139995355872], 3: [99, 590, 4494, 434, 0.5348453670640925]}, 'GaussianNB': {0: [0, 0, 1490, 4127, 0.4943002619840208], 1: [593, 5022, 2, 0, 0.5122112007389823], 2: [364, 5251, 2, 0, 0.48046955899611526], 3: [533, 5084, 0, 0, 0.46768897899897116]}, 'KNeighborsClassifier

In [36]:
with open("RAW_OUTPUT_"+datafn, 'w') as fout:
    header = ',' + ',,,,'.join("CLASS %d (%d)" % (c, np.sum(ohe_y_test[:,c])) for c in [0,1,2,3]) + '\n'
    header+= 'MODEL,' + ','.join("TP,FP,TN,FN" for c in [0,1,2,3]) + '\n'
    fout.write(header)
    for m in rawScores:
        fout.write(m+',')
        for c in rawScores[m]:
            for i in rawScores[m][c][:-1]:
                fout.write(str(i)+',')
        fout.write('\n')

In [37]:
# AUC,PPV,NPV,SEN,SPE,F1
# TP,FP,TN,FN
def calcScores(rs):
#     print('?',rs)
    auc = rs[4]
    ppv = rs[0]/(rs[0]+rs[1])
    npv = rs[2]/(rs[2]+rs[3])
    sen = rs[0]/(rs[0]+rs[3])
    spe = rs[2]/(rs[0]+rs[1])
    f1  = (2*rs[0])/((2*rs[0])+rs[1]+rs[3])
    return (auc,ppv,npv,sen,spe,f1)

In [38]:
with open("OUTPUT_"+datafn, 'w') as fout:
    header = ',AVG (%d),,,,,,' % ohe_y_test.shape[0] + ',,,,,,'.join("CLASS %d (%d)" % (c, np.sum(ohe_y_test[:,c])) for c in [0,1,2,3]) + '\n'
    header+= 'MODEL,ACC,AUC,PPV,NPV,SEN,SPE,F1,' + ','.join("AUC,PPV,NPV,SEN,SPE,F1" for c in [0,1,2,3]) + '\n'
    fout.write(header)
    for m in rawScores:
        calcedScores = [calcScores(rawScores[m][k]) for k in rawScores[m]]
        avgScores = [0 for i in calcedScores[0]]
        for i, c in enumerate(calcedScores):
            for j, _ in enumerate(c):
                avgScores[j] += calcedScores[i][j]
        for i, v in enumerate(avgScores):
            avgScores[i] = v/len(calcedScores)       
        
        fout.write(m+',')
        fout.write(str(accScores[m])+',')
        for v in avgScores:
            fout.write(str(v)+',')
        for c in calcedScores:
            for i in c:
                fout.write(str(i)+',')
        fout.write('\n')

/home/ram/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys
/home/ram/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  
/home/ram/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in long_scalars
  if __name__ == '__main__':
